In [1]:
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [3]:
data_features = pd.read_csv("features.csv")
data_features = data_features.dropna(how='all')
data_stores = pd.read_csv("stores.csv")
data_train = pd.read_csv("train.csv", low_memory = False)
data_train = data_train.dropna(how='all')
data_test = pd.read_csv("test.csv", low_memory = False)
data_test_feature = pd.read_csv("test_feature.csv")

In [4]:
data_test_feature['Date'] = pd.to_datetime(data_test_feature['Date'], format = "%Y/%m/%d")
data_features['Date'] = pd.to_datetime(data_features['Date'], format = "%m/%d/%Y")
data_train['Date'] = pd.to_datetime(data_train['Date'], format = "%m/%d/%Y")
data_test['Date'] = pd.to_datetime(data_test['Date'], format = "%m/%d/%Y")
data_test_feature['Date'] = pd.to_datetime(data_test_feature['Date'], format = "%m/%d/%Y")

In [5]:
train_merged = data_features.merge(data_stores, how = 'left')
train_merged = data_train.merge(train_merged, how = 'left')
test_merged = data_test_feature.merge(data_stores, how = 'left')
test_merged = data_test.merge(test_merged, how = 'left')
train_merged

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1.0,1.0,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1.0,2.0,2010-02-05,50605.27,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2,1.0,3.0,2010-02-05,13740.12,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
3,1.0,4.0,2010-02-05,39954.04,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
4,1.0,5.0,2010-02-05,32229.38,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400802,45.0,93.0,2012-09-07,3607.37,True,75.70,3.911,11024.45,12.8,52.63,1854.77,2055.7,191.577676,8.684,B,118221
400803,45.0,94.0,2012-09-07,3938.63,True,75.70,3.911,11024.45,12.8,52.63,1854.77,2055.7,191.577676,8.684,B,118221
400804,45.0,95.0,2012-09-07,52417.47,True,75.70,3.911,11024.45,12.8,52.63,1854.77,2055.7,191.577676,8.684,B,118221
400805,45.0,97.0,2012-09-07,7426.19,True,75.70,3.911,11024.45,12.8,52.63,1854.77,2055.7,191.577676,8.684,B,118221


In [6]:
train_merged["Day"]=train_merged['Date'].dt.day
#train_merged["Week"]=train_merged['Date'].dt.isocalendar().week
train_merged["Month"]=train_merged['Date'].dt.month
train_merged["Year"]=train_merged['Date'].dt.year
test_merged["Day"]=test_merged['Date'].dt.day
#test_merged["Week"]=test_merged['Date'].dt.isocalendar().week
test_merged["Month"]=test_merged['Date'].dt.month
test_merged["Year"]=test_merged['Date'].dt.year

In [7]:
train_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400807 entries, 0 to 400806
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         400807 non-null  float64       
 1   Dept          400807 non-null  float64       
 2   Date          400807 non-null  datetime64[ns]
 3   Weekly_Sales  400807 non-null  float64       
 4   IsHoliday     400807 non-null  object        
 5   Temperature   400807 non-null  float64       
 6   Fuel_Price    400807 non-null  float64       
 7   MarkDown1     130071 non-null  float64       
 8   MarkDown2     102591 non-null  float64       
 9   MarkDown3     118817 non-null  float64       
 10  MarkDown4     116965 non-null  float64       
 11  MarkDown5     130669 non-null  float64       
 12  CPI           400807 non-null  float64       
 13  Unemployment  400807 non-null  float64       
 14  Type          400807 non-null  object        
 15  Size          400

In [8]:
train_merged = train_merged.drop(columns = 'Date')
test_merged = test_merged.drop(columns = 'Date')

In [9]:
train_merged = train_merged.fillna(0)
train_merged = train_merged.replace(to_replace = False, value = 0)
train_merged = train_merged.replace(to_replace = True, value = 1)
train_merged = train_merged.replace(to_replace = 'A', value = int(0))
train_merged = train_merged.replace(to_replace = 'B', value = int(1))
train_merged = train_merged.replace(to_replace = 'C', value = int(2))

test_merged = test_merged.fillna(0)
test_merged = test_merged.replace(to_replace = False, value = 0)
test_merged = test_merged.replace(to_replace = True, value = 1)
test_merged = test_merged.replace(to_replace = 'A', value = int(0))
test_merged = test_merged.replace(to_replace = 'B', value = int(1))
test_merged = test_merged.replace(to_replace = 'C', value = int(2))


In [10]:
train_merged

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day,Month,Year
0,1.0,1.0,24924.50,0,42.31,2.572,0.00,0.0,0.00,0.00,0.0,211.096358,8.106,0,151315,5,2,2010
1,1.0,2.0,50605.27,0,42.31,2.572,0.00,0.0,0.00,0.00,0.0,211.096358,8.106,0,151315,5,2,2010
2,1.0,3.0,13740.12,0,42.31,2.572,0.00,0.0,0.00,0.00,0.0,211.096358,8.106,0,151315,5,2,2010
3,1.0,4.0,39954.04,0,42.31,2.572,0.00,0.0,0.00,0.00,0.0,211.096358,8.106,0,151315,5,2,2010
4,1.0,5.0,32229.38,0,42.31,2.572,0.00,0.0,0.00,0.00,0.0,211.096358,8.106,0,151315,5,2,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400802,45.0,93.0,3607.37,1,75.70,3.911,11024.45,12.8,52.63,1854.77,2055.7,191.577676,8.684,1,118221,7,9,2012
400803,45.0,94.0,3938.63,1,75.70,3.911,11024.45,12.8,52.63,1854.77,2055.7,191.577676,8.684,1,118221,7,9,2012
400804,45.0,95.0,52417.47,1,75.70,3.911,11024.45,12.8,52.63,1854.77,2055.7,191.577676,8.684,1,118221,7,9,2012
400805,45.0,97.0,7426.19,1,75.70,3.911,11024.45,12.8,52.63,1854.77,2055.7,191.577676,8.684,1,118221,7,9,2012


In [11]:
test_merged

,Store,Dept,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day,Month,Year
0,1,1,0,74.97,3.717,17212.52,7.00,18.79,1523.11,7992.72,222.582019,6.908,0,151315,14,9,2012
1,1,2,0,74.97,3.717,17212.52,7.00,18.79,1523.11,7992.72,222.582019,6.908,0,151315,14,9,2012
2,1,3,0,74.97,3.717,17212.52,7.00,18.79,1523.11,7992.72,222.582019,6.908,0,151315,14,9,2012
3,1,4,0,74.97,3.717,17212.52,7.00,18.79,1523.11,7992.72,222.582019,6.908,0,151315,14,9,2012
4,1,5,0,74.97,3.717,17212.52,7.00,18.79,1523.11,7992.72,222.582019,6.908,0,151315,14,9,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20758,45,93,0,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,1,118221,26,10,2012
20759,45,94,0,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,1,118221,26,10,2012
20760,45,95,0,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,1,118221,26,10,2012
20761,45,97,0,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,1,118221,26,10,2012


In [12]:
train_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400807 entries, 0 to 400806
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         400807 non-null  float64
 1   Dept          400807 non-null  float64
 2   Weekly_Sales  400807 non-null  float64
 3   IsHoliday     400807 non-null  int64  
 4   Temperature   400807 non-null  float64
 5   Fuel_Price    400807 non-null  float64
 6   MarkDown1     400807 non-null  float64
 7   MarkDown2     400807 non-null  float64
 8   MarkDown3     400807 non-null  float64
 9   MarkDown4     400807 non-null  float64
 10  MarkDown5     400807 non-null  float64
 11  CPI           400807 non-null  float64
 12  Unemployment  400807 non-null  float64
 13  Type          400807 non-null  int64  
 14  Size          400807 non-null  int64  
 15  Day           400807 non-null  int64  
 16  Month         400807 non-null  int64  
 17  Year          400807 non-null  int64  
dtypes: f

In [13]:
test_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20763 entries, 0 to 20762
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         20763 non-null  int64  
 1   Dept          20763 non-null  int64  
 2   IsHoliday     20763 non-null  int64  
 3   Temperature   20763 non-null  float64
 4   Fuel_Price    20763 non-null  float64
 5   MarkDown1     20763 non-null  float64
 6   MarkDown2     20763 non-null  float64
 7   MarkDown3     20763 non-null  float64
 8   MarkDown4     20763 non-null  float64
 9   MarkDown5     20763 non-null  float64
 10  CPI           20763 non-null  float64
 11  Unemployment  20763 non-null  float64
 12  Type          20763 non-null  int64  
 13  Size          20763 non-null  int64  
 14  Day           20763 non-null  int64  
 15  Month         20763 non-null  int64  
 16  Year          20763 non-null  int64  
dtypes: float64(9), int64(8)
memory usage: 2.9 MB


In [14]:
X_col = train_merged.columns.to_list()
X_col.remove('Weekly_Sales')
y_col = 'Weekly_Sales'
X = train_merged[X_col].copy()
y = train_merged[y_col].copy()

In [15]:
seed = 7
test_size = 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = seed)

In [16]:
model = XGBRegressor(objective = 'reg:squarederror', n_estimators = 600, max_depth = 8, learning_rate = 0.45) 
model.fit(X_train,y_train)
train_preds = model.predict(X_test)

In [17]:
mean_absolute_error(y_test, train_preds)

1631.7868393503854

In [18]:
accuracy = model.score(X_test, y_test)
print(accuracy)

0.9763414763150479


In [19]:
test_preds = model.predict(test_merged)
test_preds

array([19648.314 , 45007.203 , 12418.677 , ..., 51137.69  ,  7040.407 ,
        1159.7184], dtype=float32)

In [20]:
test_out = pd.DataFrame()
test_out['Id'] = test_merged['Year']
for i  in range(20763):
    test_out['Id'][i] = str(test_merged['Store'][i]) + '_' + str(test_merged['Dept'][i]) + '_' + str(test_merged['Month'][i]) + '/' + str(test_merged['Day'][i]) + '/' + str(test_merged['Year'][i])

test_out['Predicted'] = test_preds
    

print(test_out)
test_out.to_csv("Project1_Group30_Submission.csv", index = False, sep = ',')


                     Id     Predicted
0         1_1_9/14/2012  19648.314453
1         1_2_9/14/2012  45007.203125
2         1_3_9/14/2012  12418.676758
3         1_4_9/14/2012  34383.421875
4         1_5_9/14/2012  17246.988281
...                 ...           ...
20758  45_93_10/26/2012   3472.384521
20759  45_94_10/26/2012   2899.581543
20760  45_95_10/26/2012  51137.691406
20761  45_97_10/26/2012   7040.407227
20762  45_98_10/26/2012   1159.718384

[20763 rows x 2 columns]
